In [7]:
from sympy import symbols, cos, sin, exp, I, Matrix
from sympy.physics.quantum.tensorproduct import TensorProduct as kron

from sympy import pprint

from itertools import product

init_printing()

In [8]:


# Define symbolic parameters
theta = symbols('theta_1:5')  # theta_1, ..., theta_4
phi = symbols('phi_1:5')      # phi_1, ..., phi_4

# Define shorthand expressions
c = [cos(t/2) for t in theta]
s = [sin(t/2) for t in theta]
e = [exp(I*p) for p in phi]

# Define individual qubit states
psi = [Matrix([[c[j]], [e[j]*s[j]]]) for j in range(4)]

# Tensor product to get full 4-qubit state
psi_total = psi[0]
for j in range(1, 4):
    psi_total = kron(psi_total, psi[j])

# Full density matrix
rho_full = psi_total * psi_total.H

# Partial trace over qubits 3 and 4 → reduced 4x4 matrix for qubits 1 and 2
rho_12 = Matrix.zeros(4, 4)
for i3, i4 in product([0, 1], repeat=2):
    for i1, i2 in product([0, 1], repeat=2):
        for j1, j2 in product([0, 1], repeat=2):
            row = i1*8 + i2*4 + i3*2 + i4
            col = j1*8 + j2*4 + i3*2 + i4
            rho_12[i1*2 + i2, j1*2 + j2] += rho_full[row, col]

pprint(rho_12)


⎡                                        __     __                    ⎛__⎞    
⎢                         ⅈ⋅φ₃  ⅈ⋅φ₄  -ⅈ⋅φ₃  -ⅈ⋅φ₄    ⎛θ₃⎞    ⎛θ₄⎞    ⎜θ₃⎟    
⎢                        ℯ    ⋅ℯ    ⋅ℯ     ⋅ℯ     ⋅sin⎜──⎟⋅sin⎜──⎟⋅sin⎜──⎟⋅sin
⎢                                                     ⎝2 ⎠    ⎝2 ⎠    ⎝2 ⎠    
⎢                                                                             
⎢                                  __     __                            ⎛__⎞  
⎢             ⅈ⋅φ₂  ⅈ⋅φ₃  ⅈ⋅φ₄  -ⅈ⋅φ₃  -ⅈ⋅φ₄    ⎛θ₂⎞    ⎛θ₃⎞    ⎛θ₄⎞    ⎜θ₃⎟  
⎢            ℯ    ⋅ℯ    ⋅ℯ    ⋅ℯ     ⋅ℯ     ⋅sin⎜──⎟⋅sin⎜──⎟⋅sin⎜──⎟⋅sin⎜──⎟⋅s
⎢                                               ⎝2 ⎠    ⎝2 ⎠    ⎝2 ⎠    ⎝2 ⎠  
⎢                                                                             
⎢                                  __     __                            ⎛__⎞  
⎢             ⅈ⋅φ₁  ⅈ⋅φ₃  ⅈ⋅φ₄  -ⅈ⋅φ₃  -ⅈ⋅φ₄    ⎛θ₁⎞    ⎛θ₃⎞    ⎛θ₄⎞    ⎜θ₃⎟  
⎢            ℯ    ⋅ℯ    ⋅ℯ    ⋅ℯ     ⋅ℯ     ⋅sin⎜──⎟

In [10]:
from sympy import pprint

# Iterate through the reduced density matrix and print each element
for i in range(4):
    for j in range(4):
        print(f"Element ({i}, {j}):")
        print(rho_12[i, j])
        print("\n" + "-"*50 + "\n")

Element (0, 0):
exp(I*phi_3)*exp(I*phi_4)*exp(-I*conjugate(phi_3))*exp(-I*conjugate(phi_4))*sin(theta_3/2)*sin(theta_4/2)*sin(conjugate(theta_3)/2)*sin(conjugate(theta_4)/2)*cos(theta_1/2)*cos(theta_2/2)*cos(conjugate(theta_1)/2)*cos(conjugate(theta_2)/2) + exp(I*phi_3)*exp(-I*conjugate(phi_3))*sin(theta_3/2)*sin(conjugate(theta_3)/2)*cos(theta_1/2)*cos(theta_2/2)*cos(theta_4/2)*cos(conjugate(theta_1)/2)*cos(conjugate(theta_2)/2)*cos(conjugate(theta_4)/2) + exp(I*phi_4)*exp(-I*conjugate(phi_4))*sin(theta_4/2)*sin(conjugate(theta_4)/2)*cos(theta_1/2)*cos(theta_2/2)*cos(theta_3/2)*cos(conjugate(theta_1)/2)*cos(conjugate(theta_2)/2)*cos(conjugate(theta_3)/2) + cos(theta_1/2)*cos(theta_2/2)*cos(theta_3/2)*cos(theta_4/2)*cos(conjugate(theta_1)/2)*cos(conjugate(theta_2)/2)*cos(conjugate(theta_3)/2)*cos(conjugate(theta_4)/2)

--------------------------------------------------

Element (0, 1):
exp(I*phi_3)*exp(I*phi_4)*exp(-I*conjugate(phi_2))*exp(-I*conjugate(phi_3))*exp(-I*conjugate(phi_4))*

In [3]:
import numpy as np

# Example values (in radians)
vals = {
    theta[0]: np.pi/3, phi[0]: np.pi/4,
    theta[1]: np.pi/2, phi[1]: np.pi/3,
    theta[2]: np.pi/4, phi[2]: np.pi/6,
    theta[3]: np.pi/6, phi[3]: np.pi/2,
}

# Evaluate the matrix numerically with substitutions
rho_numeric = rho_12.evalf(subs=vals)

# Convert to NumPy array (complex dtype)
rho_np = np.array(rho_numeric.tolist(), dtype=np.complex128)

# Optional: print or use it
print(rho_np)


[[ 0.375     -1.95625174e-29j  0.1875    -3.24759526e-01j
   0.15309311-1.53093109e-01j -0.05603597-2.09129076e-01j]
 [ 0.1875    +3.24759526e-01j  0.375     -3.40963734e-28j
   0.20912908+5.60359670e-02j  0.15309311-1.53093109e-01j]
 [ 0.15309311+1.53093109e-01j  0.20912908-5.60359670e-02j
   0.125     +3.53351005e-28j  0.0625    -1.08253175e-01j]
 [-0.05603597+2.09129076e-01j  0.15309311+1.53093109e-01j
   0.0625    +1.08253175e-01j  0.125     +3.14276310e-27j]]
